In [2]:
import numpy as np
import pandas as pd
import os
from Bio import SeqIO
import Bio.SeqUtils.MeltingTemp as mt

from dredFISH.Utils.__init__plots import *

In [3]:
def parse_encoding(seq, option='+'):
    # P1-R-a-E-a-R-a-R-P2
    breakpoints = np.array([0, 20, 40, 41, 71, 72, 92, 93, 113, 133])
    if option == '+': # 1R-2R
        bp = breakpoints
        
        p1 = seq[bp[0]:bp[1]]
        R1 = seq[bp[1]:bp[2]]
        s1 = seq[bp[2]:bp[3]]
        E  = seq[bp[3]:bp[4]]
        s2 = seq[bp[4]:bp[5]]
        R2 = seq[bp[5]:bp[6]]
        s3 = seq[bp[6]:bp[7]]
        R3 = seq[bp[7]:bp[8]]
        p2 = seq[bp[8]:bp[9]]
        
        return E, R1, R2, R3, p1, p2, s1, s2, s3,  
        
    if option == '-': # 2R-1R
        bp = (133 - breakpoints)[::-1]
    
        p1 = seq[bp[0]:bp[1]]
        R1 = seq[bp[1]:bp[2]]
        s1 = seq[bp[2]:bp[3]]
        R2 = seq[bp[3]:bp[4]]
        s2 = seq[bp[4]:bp[5]]
        E  = seq[bp[5]:bp[6]]
        s3 = seq[bp[6]:bp[7]]
        R3 = seq[bp[7]:bp[8]]
        p2 = seq[bp[8]:bp[9]]
        
        return E, R1, R2, R3, p1, p2, s1, s2, s3,  

In [4]:
def recalc_tm(seq, fmd=0, Na=1e-5, dnac1=0, dnac2=0):
    """
    """
    res = mt.Tm_NN(seq, Na=Na, dnac1=dnac1, dnac2=dnac2)
    res = mt.chem_correction(res, fmd=fmd)
    return res
    
def get_gc(seq):
    """
    """
    return (seq.count('G') + seq.count('C'))/len(seq)

# f = "/bigstore/binfo/Probe_Sets/dredFISH_Final_Oligos.fasta"
# resall = []
# encseqs = []
# for i, record in enumerate(SeqIO.parse(f, "fasta")):
#     seq = record.seq
#     dsp = record.description
    
#     if dsp.split(' ')[4].startswith('RS'):
#         option = '-' # R[R]ER
#     else:
#         option = '+' # R[E]RR
        
#     E, R1,R2,R3, p1,p2, s1,s2,s3 = parse_encoding(seq, option=option)
#     if (str(s1),str(s2),str(s3)) == ('A', 'A', 'A'):
#         pass
#     else:
#         print(s1,s2,s3)
#         break
#     assert R1 == R2
#     assert R1 == R3
    
#     # resall.append(res)
#     encseqs.append(str(E))
    
#     # if i > 100:
#     #     break

# Get matrix; get sequences 

In [5]:
ddir = '/bigstore/GeneralStorage/fangming/projects/dredfish/res_seq'

In [6]:
f = os.path.join(ddir, 'Pvalb_Vipr2_subtype_genes.txt')
# f = os.path.join(ddir, 'DPNMF_tree_clipped_weights_Nov7_2022.csv')
mat = pd.read_csv(f, header=None) #, index_col=0)
mat

,0
0,1110032F04Rik
1,2310067E19Rik
2,9630028B13Rik
3,A730009L09Rik
4,A830019P07Rik
...,...
122,Unc5b
123,Vipr2
124,Wdfy4
125,Wnt5a


In [7]:
f = os.path.join(ddir, 'mm10_tm37-47_tp37_30bp_Oct28.csv')
pset = pd.read_csv(f)
print(pset.onscore.min(), pset.offscore.max())
pset

97.001 49.997


,chrom,start,end,seq,tm,onscore,offscore,repeat,prob,maxkmer,strand,gname,transcripts,len,gc
0,chr1,3214541,3214570,AGAAAGCGGGAATGTTTACTTGCTGTGTGG,39.76,100.0,0.0,0,0.310,0,+,Xkr4,1,30,0.466667
1,chr1,3214625,3214654,ACAAATCTTAGCTGATGGAGTGGTAAGCCC,38.59,100.0,0.0,0,0.276,0,+,Xkr4,1,30,0.466667
2,chr1,3214679,3214708,AATGGCATACACATTGCATCTGTATGCTCT,37.39,100.0,0.0,0,0.177,3,+,Xkr4,1,30,0.400000
3,chr1,3214721,3214750,TGTGTCCCAAAGTCTCTAGTAGACACATCA,37.01,100.0,0.0,0,0.234,2,+,Xkr4,1,30,0.433333
4,chr1,3214751,3214780,TTTTGACCTTGGATGGGAAGAGGGTAAGTC,38.33,100.0,0.0,0,0.381,2,+,Xkr4,1,30,0.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098716,chrY,2663507,2663536,TTTTCGGCTTCTGTAAGGCTTTTCCACCTG,39.57,100.0,0.0,0,0.302,2,+,Sry,1,30,0.466667
1098717,chrY,2663537,2663566,CATCCCAGCTGCTTGCTGATCTCTGTATTT,39.00,100.0,0.0,0,0.198,10,+,Sry,1,30,0.466667
1098718,chrY,2663567,2663596,TGCATGCTGGGATTCTGCTGGGCCAACTTG,44.74,100.0,0.0,0,0.339,0,+,Sry,1,30,0.566667
1098719,chrY,2663597,2663626,TGCCTCTCACCACGGGACCACACCATAAAT,43.00,100.0,0.0,0,0.333,0,+,Sry,1,30,0.533333


# Assign probe index
- for each probe in `pset`, assign a number (which row and which col)

### assign row index

In [8]:
gusd = mat[0].values
print(gusd.shape)
# check all genes used are in pset
gp = pset['gname'].unique() 
gusd = np.intersect1d(gusd, gp)
print(gusd.shape)

gusd_idx = pd.Series(gusd).reset_index().set_index(0)
gusd_idx

(127,)
(115,)


,index
0,
1110032F04Rik,0
9630028B13Rik,1
A830019P07Rik,2
Ackr3,3
Ak5,4
...,...
Unc5b,110
Vipr2,111
Wdfy4,112


In [9]:
psetu = pset.copy()
psetu['row_idx'] = gusd_idx.reindex(psetu['gname']).values
psetu = psetu[~psetu['row_idx'].isnull()]
psetu['row_idx'] = psetu['row_idx'].astype(int)
psetu

,chrom,start,end,seq,tm,onscore,offscore,repeat,prob,maxkmer,strand,gname,transcripts,len,gc,row_idx
9403,chr1,40282236,40282265,ACAAATGAGCCCCAGTAGCACTTTCATATT,37.13,100.000,0.0,0,0.326,0,-,Il1r1,2,30,0.400000,49
9404,chr1,40282266,40282295,AATCTCCAGCGACAGCAGAGGCACCATGAG,43.47,100.000,0.0,0,0.341,2,-,Il1r1,2,30,0.566667,49
9405,chr1,40293226,40293255,GGGGTCTTGCTGTCATTCTTGTACCAAATT,37.82,100.000,0.0,0,0.190,2,-,Il1r1,2,30,0.433333,49
9406,chr1,40293256,40293285,TGAATCCTGGAGTCCCGGTCCGCTGATATG,42.64,99.894,0.0,0,0.224,0,-,Il1r1,2,30,0.566667,49
9407,chr1,40293289,40293318,GCAGGTACAAACCAAAGATGTTCATTCTGC,37.28,100.000,0.0,0,0.192,0,-,Il1r1,2,30,0.433333,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097733,chrX,166585470,166585499,ACCTGTCACCCAGGGGAGCAGCAGAGGAAG,45.99,99.968,0.0,0,0.206,2,+,Egfl6,1,30,0.633333,29
1097734,chrX,166585532,166585561,GGGGTGGGCTCCTTTCTTGTCCTGAGCCCC,46.85,99.982,0.0,0,0.182,2,+,Egfl6,1,30,0.666667,29
1097735,chrX,166585562,166585591,TCAGGACCTACGAAACAGAGGGGCCACCGC,46.16,99.559,0.0,0,0.266,2,+,Egfl6,1,30,0.633333,29
1097736,chrX,166585592,166585621,CTCTTCTCTGCTTCGCTGGTCAGGACCTTG,41.86,100.000,0.0,0,0.408,0,+,Egfl6,1,30,0.566667,29


In [10]:
psetu['row_idx'].unique(), psetu['gname'].unique()

(array([ 49,  71,  78,   3,  83,  21,  94,  39,  26,   1,  40,  93,  72,
        110,  48,  91,  90,  13,  55,  56,   9,  98,  99,  67,  25, 108,
         46,  57,  27,  69,  79,  28,  47, 111,  53,  31,  82,  63,  87,
        113,  11, 112,   8,  74,  15,   5, 109,  97,  84, 104,  22,  51,
         66,  61, 114,   2,  38,  54,  24,  52,  75,  14,  58,  80,  95,
         33,  19,   0,  36,  41,   4, 107,  68,  35,  42,  10,  60,  59,
         85, 100,  34,  88,  30,  89, 101, 103, 102,  96,  37,  76,  81,
        106,  62,  43,  12,  44,  86,  45,  92,  50,  70,  77,   7,  23,
         17,  16,  18,   6,  65,  64,  20, 105,  32,  73,  29]),
 array(['Il1r1', 'Nyap2', 'Pid1', 'Ackr3', 'Ramp1', 'Cntnap5b', 'Slc41a1',
        'Glul', 'Ddr2', '9630028B13Rik', 'Gm2061', 'Slc30a10', 'Oprm1',
        'Unc5b', 'Igfbp3', 'Sgcd', 'Sez6', 'Car4', 'Krt12', 'Krt20',
        'C1ql1', 'Sstr2', 'St6galnac2', 'Mxra7', 'Dcxr', 'Trib2', 'Id2',
        'Lamb1', 'Dgkb', 'Nkx2-1', 'Plekhd1', 'Dpf3', 'Ifi27',

### assign col index

In [11]:
%%time

psetu['col_idx'] = 0 # all in the same
psetu

CPU times: user 1.64 ms, sys: 0 ns, total: 1.64 ms
Wall time: 1.43 ms


,chrom,start,end,seq,tm,onscore,offscore,repeat,prob,maxkmer,strand,gname,transcripts,len,gc,row_idx,col_idx
9403,chr1,40282236,40282265,ACAAATGAGCCCCAGTAGCACTTTCATATT,37.13,100.000,0.0,0,0.326,0,-,Il1r1,2,30,0.400000,49,0
9404,chr1,40282266,40282295,AATCTCCAGCGACAGCAGAGGCACCATGAG,43.47,100.000,0.0,0,0.341,2,-,Il1r1,2,30,0.566667,49,0
9405,chr1,40293226,40293255,GGGGTCTTGCTGTCATTCTTGTACCAAATT,37.82,100.000,0.0,0,0.190,2,-,Il1r1,2,30,0.433333,49,0
9406,chr1,40293256,40293285,TGAATCCTGGAGTCCCGGTCCGCTGATATG,42.64,99.894,0.0,0,0.224,0,-,Il1r1,2,30,0.566667,49,0
9407,chr1,40293289,40293318,GCAGGTACAAACCAAAGATGTTCATTCTGC,37.28,100.000,0.0,0,0.192,0,-,Il1r1,2,30,0.433333,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097733,chrX,166585470,166585499,ACCTGTCACCCAGGGGAGCAGCAGAGGAAG,45.99,99.968,0.0,0,0.206,2,+,Egfl6,1,30,0.633333,29,0
1097734,chrX,166585532,166585561,GGGGTGGGCTCCTTTCTTGTCCTGAGCCCC,46.85,99.982,0.0,0,0.182,2,+,Egfl6,1,30,0.666667,29,0
1097735,chrX,166585562,166585591,TCAGGACCTACGAAACAGAGGGGCCACCGC,46.16,99.559,0.0,0,0.266,2,+,Egfl6,1,30,0.633333,29,0
1097736,chrX,166585592,166585621,CTCTTCTCTGCTTCGCTGGTCAGGACCTTG,41.86,100.000,0.0,0,0.408,0,+,Egfl6,1,30,0.566667,29,0


In [12]:
fout = os.path.join(ddir, "encoding_probes_Pvalb_Vipr2_n115_Nov17_2022.csv")
psetu.to_csv(fout, header=True, index=False)
!chmod 444 $fout